# NLP

In [1]:
from pyspark.sql import SparkSession

In [32]:
spark = SparkSession.builder.appName('nlp').getOrCreate()
spark

In [44]:
#To load the dataset:
data = spark.read.csv("smsspamcollection/SMSSpamCollection",inferSchema=True,sep='\t')
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [45]:
#To rename the column names:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



## Clean and Prepare the Data

#### Create a new length feature

In [46]:
from pyspark.sql.functions import length

In [47]:
#To calculate the length of the sentence:
data = data.withColumn('length',length(data['text']))
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [49]:
#To check the average length of sentence for ham and spam class:
data.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



## Feature Transformations

In [50]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer

tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

In [52]:
#To import vector and vector assembler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [53]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

### The Model

We'll use Naive Bayes, but feel free to play around with this choice!

In [57]:
#To import naive bayes from pyspark
from pyspark.ml.classification import NaiveBayes

In [56]:
nb = NaiveBayes()

### Pipeline

In [58]:
from pyspark.ml import Pipeline

In [60]:
pipe = Pipeline(stages=[tokenizer,stopremove,count_vec,idf,clean_up,ham_spam_to_num])

In [61]:
cleaner = pipe.fit(data)

In [62]:
clean_data = cleaner.transform(data)

### Training and Evaluation!

In [63]:
clean_data = clean_data.select(['label','features'])

In [64]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [68]:
train,test = clean_data.randomSplit([0.7,0.3])

In [69]:
spam_predictor = nb.fit(train)

In [70]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [71]:
test_results = spam_predictor.transform(test)

In [72]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,3,9,1...|[-571.53194685442...|[0.99999999999992...|       0.0|
|  0.0|(13424,[0,1,4,50,...|[-829.49767498113...|[1.0,7.6494980655...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-882.52725244784...|[1.0,5.0752688993...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-1168.6468995735...|[1.0,2.2121809559...|       0.0|
|  0.0|(13424,[0,1,7,15,...|[-659.40259695882...|[1.0,5.6338817029...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-536.43124685228...|[1.0,1.2829855068...|       0.0|
|  0.0|(13424,[0,1,11,32...|[-872.33054330045...|[1.0,3.0554791239...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-773.24507924552...|[1.0,8.9908478180...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-1008.4157173480...|[1.0,1.3761517267...|       0.0|
|  0.0|(13424,[0

In [78]:
#Import the evaluation metrics:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [77]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting spam was: {}".format(acc))

Accuracy of model at predicting spam was: 0.925889845024347
